In [9]:
using LinearAlgebra
using DataFrames
using QuadraticModels
using SparseArrays
using BenchmarkTools
using NLPModels
using QPSReader
using SolverTools
using SolverBenchmark
using JLD2
using LDLFactorizations
using Statistics
using Quadmath

In [135]:
function starting_points(Qrows, Qcols, Qvals, Arows, Acols, Avals, b, c,
                         lvar, uvar, ilow, iupp, irng, J_augm, n_rows, n_cols, Δ_xλ)

    T = eltype(Avals)
    J_P = ldl_analyze(Symmetric(J_augm, :U))
    J_fact = ldl_factorize!(Symmetric(J_augm, :U), J_P)
    #J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :U))
#     J_fact = ldl(Symmetric(J_augm, :U))
#     J_P = J_fact.P
    Δ_xλ[n_cols+1: end] = b
    Δ_xλ = ldiv!(J_fact, Δ_xλ)
    #init_xλ2 = J_fact \ [c ; zeros(n_rows)]
    x0 = Δ_xλ[1:n_cols]
    λ0 = Δ_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(T, n_cols), zeros(T, n_cols)
    Qx, ATλ = zeros(T, n_cols), zeros(T, n_cols)
    Qx = mul_Qx_COO!(Qx, Qrows, Qcols, Qvals, x0)
    ATλ = mul_ATλ_COO!(ATλ, Arows, Acols, Avals, λ0)
    dual_val = Qx - ATλ + c
    s0_l[ilow] = @views dual_val[ilow]
    s0_u[iupp] = @views -dual_val[iupp]

    x0_m_lvar = @views x0[ilow] - lvar[ilow]
    uvar_m_x0 = @views uvar[iupp] - x0[iupp]
    if length(ilow) == 0
        δx_l1, δs_l1 = zero(T), zero(T)
    else
        δx_l1 = max(-T(1.5)*minimum(x0_m_lvar), T(1.e-2))
        δs_l1 = @views max(-T(1.5)*minimum(s0_l[ilow]), T(1.e-4))
    end

    if length(iupp) == 0
        δx_u1, δs_u1 = zero(T), zero(T)
    else
        δx_u1 = max(-T(1.5)*minimum(uvar_m_x0), T(1.e-2))
        δs_u1 = @views max(-T(1.5)*minimum(s0_u[iupp]), T(1.e-4))
    end

    x0_m_lvar .+= δx_l1
    uvar_m_x0 .+= δx_u1
    s0_l1 = @views s0_l[ilow] .+ δs_l1
    s0_u1 = @views s0_u[iupp] .+ δs_u1
    xs_l1, xs_u1 = s0_l1' * x0_m_lvar, s0_u1' * uvar_m_x0
    if length(ilow) == 0
        δx_l2, δs_l2 = zero(T), zero(T)
    else
        δx_l2 = δx_l1 + xs_l1 / sum(s0_l1) / 2
        δs_l2 = @views δs_l1 + xs_l1 / sum(x0_m_lvar) / 2
    end
    if length(iupp) == 0
        δx_u2, δs_u2 = zero(T), zero(T)
    else
        δx_u2 = δx_u1 + xs_u1 / sum(s0_u1) / 2
        δs_u2 = @views δs_u1 + xs_u1 / sum(uvar_m_x0) / 2
    end
    δx = max(δx_l2, δx_u2)
    δs = max(δs_l2, δs_u2)
    x0[ilow] .+= δx
    x0[iupp] .-= δx
    s0_l[ilow] = @views s0_l[ilow] .+ δs
    s0_u[iupp] = @views s0_u[iupp] .+ δs

    @inbounds @simd for i in irng
        if lvar[i] >= x0[i]
            x0[i] = lvar[i] + T(1e-4)
        end
        if x0[i] >= uvar[i]
            x0[i] = uvar[i] - T(1e-4)
        end
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2
        end
    end

    x0_m_lvar .= @views x0[ilow] .- lvar[ilow]
    uvar_m_x0 .= @views uvar[iupp] .- x0[iupp]

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert @views all(s0_l[ilow] .> zero(T)) && all(s0_u[iupp] .> zero(T))

    return x0, λ0, s0_l, s0_u, J_fact, J_P, Qx, ATλ, x0_m_lvar, uvar_m_x0, Δ_xλ
end


starting_points (generic function with 1 method)

In [136]:
function compute_α_dual(v, dir_v)
    n = length(v)
    T = eltype(v)
    if n == 0
        return one(T)
    end
    α = one(T)
    @inbounds @simd for i=1:n
        if dir_v[i] < zero(T)
            α_new = -v[i] * T(0.999) / dir_v[i]
            if α_new < α
                α = α_new
            end
        end
    end
    return α
end



function compute_α_primal(v, dir_v, lvar, uvar)
    n = length(v)
    T = eltype(v)
    α_l, α_u = one(T), one(T)
    @inbounds @simd for i=1:n
        if dir_v[i] > zero(T)
            α_u_new = (uvar[i] - v[i]) * T(0.999) / dir_v[i]
            if α_u_new < α_u
                α_u = α_u_new
            end
        elseif dir_v[i] < zero(T)
            α_l_new = (lvar[i] - v[i]) * T(0.999) / dir_v[i]
            if α_l_new < α_l
                α_l = α_l_new
            end
        end
    end
    return min(α_l, α_u)
end



function compute_μ(x_m_lvar, uvar_m_x, s_l, s_u, nb_low, nb_upp)
    return (s_l' * x_m_lvar + s_u' * uvar_m_x) / (nb_low + nb_upp)
end



compute_μ (generic function with 1 method)

In [137]:
function solve_augmented_system_aff!(J_fact, Δ_aff, Δ_xλ, rc, rb, x_m_lvar, uvar_m_x,
                                     s_l, s_u, ilow, iupp,  n_cols, n_rows, n_low)

    Δ_xλ[1:n_cols] .= .-rc
    Δ_xλ[n_cols+1:end] .= .-rb
    Δ_xλ[ilow] += @views s_l[ilow]
    Δ_xλ[iupp] -= @views s_u[iupp]

    Δ_xλ = ldiv!(J_fact, Δ_xλ)

    Δ_aff[1:n_cols+n_rows] = Δ_xλ
    Δ_aff[n_cols+n_rows+1:n_cols+n_rows+n_low] = @views -s_l[ilow] - s_l[ilow].*Δ_xλ[1:n_cols][ilow]./x_m_lvar
    Δ_aff[n_cols+n_rows+n_low+1:end] = @views -s_u[iupp] + s_u[iupp].*Δ_xλ[1:n_cols][iupp]./uvar_m_x
    return Δ_aff
end

function solve_augmented_system_cc!(J_fact, Δ_cc, Δ_xλ ,Δ_aff, σ, μ, x_m_lvar, uvar_m_x,
                                    rxs_l, rxs_u, s_l, s_u, ilow, iupp, n_cols, n_rows, n_low)


    rxs_l .= @views (-σ*μ .+ Δ_aff[1:n_cols][ilow].*Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
    rxs_u .= @views σ*μ .+ Δ_aff[1:n_cols][iupp].*Δ_aff[n_rows+n_cols+n_low+1: end]

    Δ_xλ .= zero(eltype(Δ_xλ))
    Δ_xλ[ilow] .+= rxs_l./x_m_lvar
    Δ_xλ[iupp] .+= rxs_u./uvar_m_x

    Δ_xλ = ldiv!(J_fact, Δ_xλ)

    Δ_cc[1:n_cols+n_rows] = Δ_xλ
    Δ_cc[n_cols+n_rows+1:n_cols+n_rows+n_low] .= @views .-(rxs_l.+s_l[ilow].*Δ_xλ[1:n_cols][ilow])./x_m_lvar
    Δ_cc[n_cols+n_rows+n_low+1:end] .= @views (rxs_u.+s_u[iupp].*Δ_xλ[1:n_cols][iupp])./uvar_m_x
    return Δ_cc
end


solve_augmented_system_cc! (generic function with 1 method)

In [138]:
function mul_Qx_COO!(Qx, Qrows, Qcols, Qvals, x)
    # right mutiplication for sparse COO symetric matrix M: res=Mv
    Qx .= zero(eltype(Qx))
    @inbounds @simd for i=1:length(Qcols)
        Qx[Qrows[i]] += Qvals[i] * x[Qcols[i]]
        if Qrows[i] != Qcols[i]
            Qx[Qcols[i]] += Qvals[i]*x[Qrows[i]]
        end
    end
    return Qx
end

function mul_ATλ_COO!(ATλ, Arows, Acols, Avals, λ)
    ATλ .= zero(eltype(ATλ))
    @inbounds @simd for i=1:length(Acols)
        ATλ[Acols[i]] += Avals[i] * λ[Arows[i]]
    end
    return ATλ
end

function mul_Ax_COO!(Ax, Arows, Acols, Avals, x)
    Ax .= zero(eltype(Ax))
    @inbounds @simd for i=1:length(Acols)
        Ax[Arows[i]] += Avals[i] * x[Acols[i]]
    end
    return Ax
end



function get_norm_rc!(v, A_i, Avals, n_v, n)
    T = eltype(v)
    v .= zero(T)
    @inbounds @simd for j=1:n
        if abs(Avals[j]) > v[A_i[j]]
            v[A_i[j]] = abs(Avals[j])
        end
#         v[A_i[j]] += Avals[j]^2  #2-norm
    end

    v = sqrt.(v)
    @inbounds @simd for i=1:n_v
        if v[i] == zero(T)
            v[i] = one(T)
        end
    end
    return v
end

function mul_A_D1_D2!(Arows, Acols, Avals, d1, d2, r, c, n_rows, n_cols, n)
    @inbounds @simd for i=1:n
        Avals[i] /= r[Arows[i]] * c[Acols[i]]
    end
    d1 ./= r
    d2 ./= c
    return Arows, Acols, Avals, d1, d2
end

function mul_Q_D!(Qrows, Qcols, Qvals, d, c, n_cols, n)
    @inbounds @simd for i=1:n
        Qvals[i] /= c[Qrows[i]] * c[Qcols[i]]
    end
    d ./= c
    return Qrows, Qcols, Qvals, d
end

function scaling_Ruiz!(Arows, Acols, Avals, Qrows, Qcols, Qvals, c, b, lvar, uvar,
                       n_rows, n_cols, ϵ; max_iter = 100)
    n = length(Arows)
    T = eltype(Avals)
    d1, d2 = ones(T, n_rows), ones(T, n_cols)
    r_k, c_k = zeros(T, n_rows), zeros(T, n_cols)

    r_k = get_norm_rc!(r_k, Arows, Avals, n_rows, n)
    c_k = get_norm_rc!(c_k, Acols, Avals, n_cols, n)
    convergence = maximum(abs.(one(T) .- r_k)) <= ϵ && maximum(abs.(one(T) .- c_k)) <= ϵ
    Arows, Acols, Avals, d1, d2 = mul_A_D1_D2!(Arows, Acols, Avals, d1, d2,
                                               r_k, c_k, n_rows, n_cols, n)
    k = 1
    while !convergence && k < max_iter
        r_k = get_norm_rc!(r_k, Arows, Avals, n_rows, n)
        c_k = get_norm_rc!(c_k, Acols, Avals, n_cols, n)
        convergence = maximum(abs.(one(T) .- r_k)) <= ϵ && maximum(abs.(one(T) .- c_k)) <= ϵ
        Arows, Acols, Avals, d1, d2 = mul_A_D1_D2!(Arows, Acols, Avals, d1, d2,
                                                   r_k, c_k, n_rows, n_cols, n)
        k += 1
    end

    n_Q = length(Qrows)
    @inbounds @simd for i=1:n_Q
        Qvals[i] *= d2[Qrows[i]] * d2[Qcols[i]]
    end
    b .*= d1
    c .*= d2
    lvar ./= d2
    uvar ./= d2

    # scaling Q (symmetric)
    d3 = ones(T, n_cols)
    c_k .= zero(T)
    c_k = get_norm_rc!(c_k, Qcols, Qvals, n_cols, n_Q)
    convergence = maximum(abs.(one(T) .- c_k)) <= ϵ
    Qrows, Qcols, Qvals, d3 = mul_Q_D!(Qrows, Qcols, Qvals, d3, c_k, n_cols, n_Q)
    k = 1
    while !convergence && k < max_iter
        c_k = get_norm_rc!(c_k, Qcols, Qvals, n_cols, n_Q)
        convergence = maximum(abs.(one(T) .- c_k)) <= ϵ
        Qrows, Qcols, Qvals, d3 = mul_Q_D!(Qrows, Qcols, Qvals, d3, c_k, n_cols, n_Q)
        k += 1
    end

    for i=1:n
        Avals[i] *= d3[Acols[i]]
    end
    c .*= d3
    lvar ./= d3
    uvar ./= d3

    return Arows, Acols, Avals, Qrows, Qcols, Qvals, c, b, lvar, uvar, d1, d2, d3
end

scaling_Ruiz! (generic function with 1 method)

In [139]:
function get_diag_sparseCSC(M; tri=:U)
    # get diagonal index of M.nzval
    # we assume all columns of M are non empty, and M triangular (:L or :U)
    @assert tri ==:U || tri == :L
    T = eltype(M)
    n = length(M.rowval)
    diagind = zeros(Int, M.m) # square matrix
    index = M.rowval[1] # 1
    if tri == :U
        for i=1:M.m
            diagind[i] = M.colptr[i+1] - 1
        end
    else
        for i=1:M.m
            diagind[i] = M.colptr[i]
        end
    end
    return diagind
end

function get_diag_sparseCOO(Qrows, Qcols, Qvals, n_cols)
    # get diagonal index of M.nzval
    # we assume all columns of M are non empty, and M triangular (:L or :U)
    T = eltype(Qvals)
    n = length(Qrows)
    diagval = zeros(T, n_cols)
    for i=1:n
        if Qrows[i] == Qcols[i]
            diagval[Qrows[i]] = Qvals[i]
        end
    end

    return diagval
end


get_diag_sparseCOO (generic function with 1 method)

In [392]:
function iter_mehrotraPC!(x, λ, s_l, s_u, x_m_lvar, uvar_m_x, lvar, uvar, 
                          ilow, iupp, n_rows, n_cols,n_low, n_upp, 
                          Arows, Acols, Avals, Qrows, Qcols, Qvals, c0, 
                          c, b, rc, rb, rcNorm, rbNorm, tol_rb, tol_rc, 
                          Qx, ATλ, Ax, xTQx_2, cTx, pri_obj, dual_obj, 
                          pdd, l_pdd, mean_pdd, n_Δx, small_Δx, small_μ, 
                          Δt, tired, optimal, μ, k, ρ, δ, ρ_min, δ_min, 
                          J_augm, J_fact, J_P, diagind_J, diag_Q, tmp_diag, 
                          Δ_aff, Δ_cc, Δ, Δ_xλ, s_l_αΔ_aff, s_u_αΔ_aff, 
                          x_m_l_αΔ_aff, u_m_x_αΔ_aff, rxs_l, rxs_u, 
                          max_iter, ϵ_pdd, ϵ_μ, ϵ_rc, ϵ_rb, tol_Δx,  
                          start_time, max_time, c_catch, c_pdd, display)
    T = eltype(Avals)
    
        while k<max_iter && !optimal && !tired # && !small_μ && !small_μ

            # Affine scaling direction
        tmp_diag .= -ρ
        tmp_diag[ilow] .-= @views s_l[ilow] ./ x_m_lvar
        tmp_diag[iupp] .-= @views s_u[iupp] ./ uvar_m_x

        J_augm.nzval[view(diagind_J,1:n_cols)] .= @views tmp_diag .- diag_Q
        J_augm.nzval[view(diagind_J, n_cols+1:n_rows+n_cols)] .= δ

#         J_fact = ldl_factorize!(Symmetric(J_augm, :U), J_P)
        J_fact = try ldl_factorize!(Symmetric(J_augm, :U), J_P)
        catch
            println("error ", k)
            if T == Float32
                break
                break
            end
            if c_pdd == 0
                δ *= T(1e2)
                δ_min *= T(1e2)
                ρ *= T(1e5)
                ρ_min *= T(1e5)
            elseif c_pdd == 0 && c_catch != 0
                δ *= T(1e2)
                δ_min *= T(1e2)
                ρ *= T(1e1)
                ρ_min *= T(1e1)
            else
                δ *= T(1e6)
                δ_min *= T(1e6)
                ρ *= T(1e5)
                ρ_min *= T(1e5)
            end
            c_catch += 1
            tmp_diag .= -ρ
            tmp_diag[ilow] .-= @views s_l[ilow] ./ x_m_lvar
            tmp_diag[iupp] .-= @views s_u[iupp] ./ uvar_m_x
            J_augm.nzval[view(diagind_J,1:n_cols)] .= @views tmp_diag .- diag_Q
            J_augm.nzval[view(diagind_J, n_cols+1:n_rows+n_cols)] .= δ
            J_fact = ldl_factorize!(Symmetric(J_augm, :U), J_P)
        end

        if c_catch >= 3
            break
        end
        
        if k==3 && T==Float32
            println("float32  ", k)
        end
        
        Δ_aff = solve_augmented_system_aff!(J_fact, Δ_aff, Δ_xλ, rc, rb, x_m_lvar, uvar_m_x,
                                            s_l, s_u, ilow, iupp,  n_cols, n_rows, n_low)
        α_aff_pri = @views compute_α_primal(x, Δ_aff[1:n_cols], lvar, uvar)
        α_aff_dual_l = @views compute_α_dual(s_l[ilow], Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_aff_dual_u = @views compute_α_dual(s_u[iupp], Δ_aff[n_rows+n_cols+n_low+1:end])

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        α_aff_dual_final = min(α_aff_dual_l, α_aff_dual_u)

        x_m_l_αΔ_aff .= @views x_m_lvar .+ α_aff_pri .* Δ_aff[1:n_cols][ilow]
        u_m_x_αΔ_aff .= @views uvar_m_x .- α_aff_pri .* Δ_aff[1:n_cols][iupp]
        s_l_αΔ_aff .= @views s_l[ilow] .+ α_aff_dual_final .* Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low]
        s_u_αΔ_aff .= @views s_u[iupp] .+ α_aff_dual_final .*  Δ_aff[n_rows+n_cols+n_low+1: end]

        μ_aff = compute_μ(x_m_l_αΔ_aff, u_m_x_αΔ_aff, s_l_αΔ_aff, s_u_αΔ_aff,
                          n_low, n_upp)

        σ = (μ_aff / μ)^3

        # corrector and centering step

        Δ_cc = solve_augmented_system_cc!(J_fact, Δ_cc, Δ_xλ , Δ_aff, σ, μ,x_m_lvar, uvar_m_x,
                                          rxs_l, rxs_u, s_l, s_u, ilow, iupp, n_cols, n_rows, n_low)

        Δ .= Δ_aff .+ Δ_cc # final direction

        α_pri = @views compute_α_primal(x, Δ[1:n_cols], lvar, uvar)
        α_dual_l = @views compute_α_dual(s_l[ilow], Δ[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_dual_u = @views compute_α_dual(s_u[iupp], Δ[n_rows+n_cols+n_low+1: end])

        α_dual_final = min(α_dual_l, α_dual_u)

        # new parameters
        x .= @views x .+ α_pri .* Δ[1:n_cols]
        λ .= @views λ .+ α_dual_final .* Δ[n_cols+1: n_rows+n_cols]
        s_l[ilow] .= @views s_l[ilow] .+ α_dual_final .* Δ[n_rows+n_cols+1: n_rows+n_cols+n_low]
        s_u[iupp] .= @views s_u[iupp] .+ α_dual_final .* Δ[n_rows+n_cols+n_low+1: end]
        n_Δx = @views α_pri * norm(Δ[1:n_cols])
        x_m_lvar .= @views x[ilow] .- lvar[ilow]
        uvar_m_x .= @views uvar[iupp] .- x[iupp]

        if zero(T) in x_m_lvar
            for i=1:n_low
                if x_m_lvar[i] == zero(T)
                    x_m_lvar[i] = eps(T)^2
                end
            end
        end
        if zero(T) in uvar_m_x
            for i=1:n_upp
                if uvar_m_x[i] == zero(T)
                    uvar_m_x[i] = eps(T)^2
                end
            end
        end

        μ = @views compute_μ(x_m_lvar, uvar_m_x, s_l[ilow], s_u[iupp],
                             n_low, n_upp)


        Qx = mul_Qx_COO!(Qx, Qrows, Qcols, Qvals, x)
        xTQx_2 =  x' * Qx / 2
        ATλ = mul_ATλ_COO!(ATλ, Arows, Acols, Avals, λ)
        Ax = mul_Ax_COO!(Ax, Arows, Acols, Avals, x)
        cTx = c' * x
        pri_obj = xTQx_2 + cTx + c0
        dual_obj = b' * λ - xTQx_2 + view(s_l,ilow)'*view(lvar,ilow) -
                    view(s_u,iupp)'*view(uvar,iupp) +c0

        rb .= Ax .- b
        rc .= ATλ .-Qx .+ s_l .- s_u .- c
        # update stopping criterion values:

        pdd = abs(pri_obj - dual_obj ) / (one(T) + abs(pri_obj))
#         rcNorm, rbNorm = norm(rc), norm(rb)
#         xNorm = norm(x)
#         λNorm = norm(λ)
#         optimal = pdd < ϵ_pdd && rbNorm < ϵ_rb * max(1, bNorm + ANorm * xNorm) &&
#                     rcNorm < ϵ_rc * max(1, cNorm + QNorm * xNorm + ANorm * λNorm)
        rcNorm, rbNorm = norm(rc, Inf), norm(rb, Inf)
        optimal = pdd < ϵ_pdd && rbNorm < tol_rb && rcNorm < tol_rc
        small_Δx, small_μ = n_Δx < tol_Δx, μ < ϵ_μ
        k += 1

        l_pdd[k%6+1] = pdd
        mean_pdd = mean(l_pdd)

        if T == Float64 && k > 10  && mean_pdd!=zero(T) && std(l_pdd./mean_pdd) < 1e-2 && c_pdd < 5
#             println("pdd  ", k)
            δ_min /= 10
            δ /= 10
            c_pdd += 1
        end

        if T == Float64 && k>10 && c_catch == 0 && 
                @views minimum(J_augm.nzval[view(diagind_J,1:n_cols)]) < -one(T) / δ / T(1e-6)
#             println("diagJ iter ", k) 
            δ /= 100
            δ_min /= 100
            c_pdd += 1
        end

        if δ >= δ_min
            δ /= 10
            #J_augmvals[end-n_cols-n_rows+1:end-n_cols] .= δ
        end
        if ρ >= ρ_min
            ρ /= 10
        end

        Δt = time() - start_time
        tired = Δt > max_time

        if display == true
            @info log_row(Any[k, pri_obj, pdd, rbNorm, rcNorm, n_Δx, α_pri, α_dual_final, μ])
        end
#         println("iter ", k)
#         println("min diagJ = ", minimum(J_augm.nzval[view(diagind_J,1:n_cols)]))
#         println("δ = ", δ, "    ρ = ", ρ) 
    end

    return x, λ, s_l, s_u, x_m_lvar, uvar_m_x, rc, rb, 
                rcNorm, rbNorm, Qx, ATλ, Ax, xTQx_2, cTx, pri_obj, dual_obj, 
                pdd, l_pdd, mean_pdd, n_Δx, Δt, tired, optimal, μ, k, 
                ρ, δ, ρ_min, δ_min, J_augm, J_fact, c_catch, c_pdd 
end

iter_mehrotraPC! (generic function with 6 methods)

In [401]:
function mehrotraPCQuadBounds(QM0; max_iter=200, ϵ_pdd=1e-8, ϵ_rb=1e-6, ϵ_rc=1e-6,
                              tol_Δx=1e-16, ϵ_μ=1e-9, max_time=120., scaling=true,
                              display=true)
    
    start_time = time()
    elapsed_time = 0.0
    QM = SlackModel(QM0)


    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    n_cols = length(lvar)
    Oc = zeros(n_cols)
    ilow, iupp = [QM.meta.ilow; QM.meta.irng], [QM.meta.iupp; QM.meta.irng] # finite bounds index
    irng = QM.meta.irng
    ifix = QM.meta.ifix
    c = grad(QM, Oc)
    A = jac(QM, Oc)
    A = dropzeros!(A)
    T = eltype(A)
    Arows, Acols, Avals = findnz(A)
    n_rows, n_cols = size(A)
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    Q = hess(QM, Oc)  # lower triangular
    Q = dropzeros!(Q)
    Qrows, Qcols, Qvals = findnz(Q)
    c0 = obj(QM, Oc)

    if scaling
        Arows, Acols, Avals, Qrows, Qcols, Qvals,
        c, b, lvar, uvar, d1, d2, d3 = scaling_Ruiz!(Arows, Acols, Avals, Qrows, Qcols, Qvals,
                                                     c, b, lvar, uvar, n_rows, n_cols, T(1.0e-3))
    end
    
#     cNorm = norm(c)
#     bNorm = norm(b)
#     ANorm = norm(Avals)  # Frobenius norm after scaling; could be computed while scaling?
#     QNorm = norm(Qvals)

    n_low, n_upp = length(ilow), length(iupp) # number of finite constraints
    
    #change types
    T = Float32
    Qvals32 = Array{T}(Qvals)
    Avals32 = Array{T}(Avals)
    c32 = Array{T}(c)
    c032 = T(c0)
    b32 = Array{T}(b)
    lvar32 = Array{T}(lvar)
    uvar32 = Array{T}(uvar)
    ϵ_pdd32 = T(1e-1)
    ϵ_rb32 = T(1e-2)
    ϵ_rc32 = T(1e-2)
    tol_Δx32 = T(tol_Δx)
    ϵ_μ32 = T(ϵ_μ)
    
    # init regularization values
    ρ, δ = T(sqrt(eps())*1e5), T(sqrt(eps())*1e5) # 1e6, 1e-1 ok
#     ρ, δ = T(1.5e-3), T(1.5e-3) 
#     ρ_min, δ_min = T(sqrt(eps())*1e-5), T(sqrt(eps()))
    ρ_min, δ_min = T(sqrt(eps(T))*1e0), T(sqrt(eps(T))*1e0)
#     ρ_min, δ_min = T(1.5e-13), T(1.5e-8)
    c_catch = zero(Int) # to avoid endless loop
    c_pdd = zero(Int) # avoid too small δ_min

    J_augmrows = vcat(Qcols, Acols, n_cols+1:n_cols+n_rows, 1:n_cols)
    J_augmcols = vcat(Qrows, Arows.+n_cols, n_cols+1:n_cols+n_rows, 1:n_cols)
    tmp_diag = -T(1.0e0)/2 .* ones(T, n_cols)
    J_augmvals = vcat(-Qvals32, Avals32, δ.*ones(T, n_rows), tmp_diag)
    J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)
    diagind_J = get_diag_sparseCSC(J_augm)
    diag_Q = get_diag_sparseCOO(Qrows, Qcols, Qvals32, n_cols)

    k = 0
    Δ_aff = zeros(T, n_cols+n_rows+n_low+n_upp)
    Δ_cc = zeros(T, n_cols+n_rows+n_low+n_upp)
    Δ = zeros(T, n_cols+n_rows+n_low+n_upp)
    Δ_xλ = zeros(T, n_cols+n_rows)

    x, λ, s_l, s_u, J_fact, J_P, Qx, ATλ,
    x_m_lvar, uvar_m_x, Δ_xλ = @views starting_points(Qrows, Qcols, Qvals32, Arows, Acols, Avals32,
                                                      b32, c32, lvar32, uvar32, ilow, iupp, QM.meta.irng,
                                                      J_augm , n_rows, n_cols, Δ_xλ)


    Qx = mul_Qx_COO!(Qx, Qrows, Qcols, Qvals32, x)
    ATλ = mul_ATλ_COO!(ATλ, Arows, Acols, Avals32, λ)
    Ax = zeros(T,  n_rows)
    Ax = mul_Ax_COO!(Ax, Arows, Acols, Avals32, x)
    rb = Ax - b32
    rc = -Qx + ATλ + s_l - s_u - c32

    x_m_lvar .= @views x[ilow] .- lvar32[ilow]
    uvar_m_x .= @views uvar32[iupp] .- x[iupp]
    μ = @views compute_μ(x_m_lvar, uvar_m_x, s_l[ilow], s_u[iupp], n_low, n_upp)

    x_m_l_αΔ_aff = zeros(T, n_low) # x-lvar + αΔ_aff
    u_m_x_αΔ_aff = zeros(T, n_upp) # uvar-x + αΔ_aff
    s_l_αΔ_aff = zeros(T, n_low) # s_l + αΔ_aff
    s_u_αΔ_aff = zeros(T, n_upp) # s_l + αΔ_aff
    rxs_l, rxs_u = zeros(T, n_low), zeros(T, n_upp)

    # stopping criterion
    xTQx_2 = x' * Qx / 2
    cTx = c32' * x
    pri_obj = xTQx_2 + cTx + c032
    dual_obj = b32' * λ - xTQx_2 + view(s_l,ilow)'*view(lvar32,ilow) -
                    view(s_u,iupp)'*view(uvar32,iupp) +c032
    pdd = abs(pri_obj - dual_obj ) / (one(T) + abs(pri_obj))
#     rcNorm, rbNorm = norm(rc), norm(rb)
#     optimal = pdd < ϵ_pdd && rbNorm < ϵ_rb && rcNorm < ϵ_rc
    rcNorm, rbNorm = norm(rc, Inf), norm(rb, Inf)
    tol_rb32, tol_rc32 = ϵ_rb32*(one(T) + rbNorm), ϵ_rc32*(one(T) + rcNorm)
    tol_rb, tol_rc = ϵ_rb*(one(Float64) + Float64(rbNorm)), ϵ_rc*(one(Float64) + Float64(rcNorm))
    optimal = pdd < ϵ_pdd32 && rbNorm < tol_rb32 && rcNorm < tol_rc32
    
    l_pdd = zeros(T, 6)
    mean_pdd = one(T)

    n_Δx = zero(T)
    small_Δx, small_μ = false, μ < ϵ_μ32
    Δt = time() - start_time
    tired = Δt > max_time

    # display
    if display == true
        @info log_header([:k, :pri_obj, :pdd, :rbNorm, :rcNorm, :n_Δx, :α_pri, :α_du, :μ],
                         [Int, T, T, T, T, T, T, T, T, T],
                         hdr_override=Dict(:k => "iter", :pri_obj => "obj", :pdd => "rgap",
                                           :rbNorm => "‖rb‖", :rcNorm => "‖rc‖",
                                           :n_Δx => "‖Δx‖"))
        @info log_row(Any[k, pri_obj, pdd, rbNorm, rcNorm, n_Δx, zero(T), zero(T), μ])
    end
    
    # iters Float 32
    x, λ, s_l, s_u, x_m_lvar, uvar_m_x, 
        rc, rb, rcNorm, rbNorm, Qx, ATλ, 
        Ax, xTQx_2, cTx, pri_obj, dual_obj, 
        pdd, l_pdd, mean_pdd, n_Δx, Δt, 
        tired, optimal, μ, k, ρ, δ, 
        ρ_min, δ_min, J_augm, J_fact, 
        c_catch, c_pdd  = iter_mehrotraPC!(x, λ, s_l, s_u, x_m_lvar, uvar_m_x, lvar32, uvar32, 
                                          ilow, iupp, n_rows, n_cols,n_low, n_upp, 
                                          Arows, Acols, Avals32, Qrows, Qcols, Qvals32, c032, 
                                          c32, b32, rc, rb, rcNorm, rbNorm, tol_rb32, tol_rc32, 
                                          Qx, ATλ, Ax, xTQx_2, cTx, pri_obj, dual_obj, 
                                          pdd, l_pdd, mean_pdd, n_Δx, small_Δx, small_μ, 
                                          Δt, tired, optimal, μ, k, ρ, δ, ρ_min, δ_min, 
                                          J_augm, J_fact, J_P, diagind_J, diag_Q, tmp_diag, 
                                          Δ_aff, Δ_cc, Δ, Δ_xλ, s_l_αΔ_aff, s_u_αΔ_aff, 
                                          x_m_l_αΔ_aff, u_m_x_αΔ_aff, rxs_l, rxs_u, 
                                          20, ϵ_pdd32, ϵ_μ32, ϵ_rc32, ϵ_rb32, tol_Δx32,  
                                          start_time, max_time, c_catch, c_pdd, display)

    
    
    # conversions to Float64
    T = Float64
    x, λ, s_l, s_u = convert(Array{T}, x), convert(Array{T}, λ), convert(Array{T}, s_l), convert(Array{T}, s_u)
    x_m_lvar, uvar_m_x = convert(Array{T}, x_m_lvar), convert(Array{T}, uvar_m_x)
    rc, rb = convert(Array{T}, rc), convert(Array{T}, rb)
    rcNorm, rbNorm = convert(T, rcNorm), convert(T, rbNorm)
    Qx, ATλ, Ax = convert(Array{T}, Qx), convert(Array{T}, ATλ), convert(Array{T}, Ax)
    xTQx_2, cTx = convert(T, xTQx_2), convert(T, cTx)
    pri_obj, dual_obj = convert(T, pri_obj), convert(T, dual_obj)
    pdd, l_pdd, mean_pdd = convert(T, pdd), convert(Array{T}, l_pdd), convert(T, mean_pdd)
    n_Δx, μ = convert(T, n_Δx), convert(T, μ)
    ρ, δ = convert(T, ρ), convert(T, δ)
    J_augm = convert(SparseMatrixCSC{T,Int64}, J_augm)
    #J_fact = convert(LDLFactorizations.LDLFactorization{T,Int64,Int64,Int64}, J_fact)
    
    J_P = LDLFactorizations.LDLFactorization(J_P.__analyzed, J_P.__factorized, J_P.__upper,
                              J_P.n, J_P.parent, J_P.Lnz, J_P.flag, J_P.P, 
                              J_P.pinv, J_P.Lp, J_P.Cp, J_P.Ci, J_P.Li,
                              Array{T}(J_P.Lx), Array{T}(J_P.d), Array{T}(J_P.Y), J_P.pattern)
    J_fact = LDLFactorizations.LDLFactorization(J_fact.__analyzed, J_fact.__factorized, J_fact.__upper,
                              J_fact.n, J_fact.parent, J_fact.Lnz, J_fact.flag, J_fact.P, 
                              J_fact.pinv, J_fact.Lp, J_fact.Cp, J_fact.Ci, J_fact.Li,
                              Array{T}(J_fact.Lx), Array{T}(J_fact.d), Array{T}(J_fact.Y), J_fact.pattern)    

    
    Δ_aff, Δ_cc, Δ = convert(Array{T}, Δ_aff), convert(Array{T}, Δ_cc), convert(Array{T}, Δ)
    Δ_xλ, rxs_l, rxs_u = convert(Array{T}, Δ_xλ), convert(Array{T}, rxs_l), convert(Array{T}, rxs_u)
    s_l_αΔ_aff, s_u_αΔ_aff = convert(Array{T}, s_l_αΔ_aff), convert(Array{T}, s_u_αΔ_aff)
    x_m_l_αΔ_aff, u_m_x_αΔ_aff = convert(Array{T}, x_m_l_αΔ_aff), convert(Array{T}, u_m_x_αΔ_aff)
    
    ρ_min, δ_min = T(sqrt(eps())*1e-5), T(sqrt(eps()))
    
    
    optimal = pdd < ϵ_pdd && rbNorm < tol_rb && rcNorm < tol_rc
    println("iter float32 = ", k)
    # iters Float64
    x, λ, s_l, s_u, x_m_lvar, uvar_m_x, 
        rc, rb, rcNorm, rbNorm, Qx, ATλ, 
        Ax, xTQx_2, cTx, pri_obj, dual_obj, 
        pdd, l_pdd, mean_pdd, n_Δx, Δt, 
        tired, optimal, μ, k, ρ, δ, 
        ρ_min, δ_min, J_augm, J_fact, 
        c_catch, c_pdd  = iter_mehrotraPC!(x, λ, s_l, s_u, x_m_lvar, uvar_m_x, lvar, uvar, 
                                          ilow, iupp, n_rows, n_cols,n_low, n_upp, 
                                          Arows, Acols, Avals, Qrows, Qcols, Qvals, c0, 
                                          c, b, rc, rb, rcNorm, rbNorm, tol_rb, tol_rc, 
                                          Qx, ATλ, Ax, xTQx_2, cTx, pri_obj, dual_obj, 
                                          pdd, l_pdd, mean_pdd, n_Δx, small_Δx, small_μ, 
                                          Δt, tired, optimal, μ, k, ρ, δ, ρ_min, δ_min, 
                                          J_augm, J_fact, J_P, diagind_J, diag_Q, tmp_diag, 
                                          Δ_aff, Δ_cc, Δ, Δ_xλ, s_l_αΔ_aff, s_u_αΔ_aff, 
                                          x_m_l_αΔ_aff, u_m_x_αΔ_aff, rxs_l, rxs_u, 
                                          max_iter, ϵ_pdd, ϵ_μ, ϵ_rc, ϵ_rb, tol_Δx,  
                                          start_time, max_time, c_catch, c_pdd, display)   
    
    
    if k>= max_iter
        status = :max_iter
    elseif tired
        status = :max_time
    elseif optimal
        status = :acceptable
    else
        status = :unknown
    end

    if scaling
#         d1 = convert(Array{T}, d1)
#         d2 = convert(Array{T}, d2)
#         d3 = convert(Array{T}, d3)
        x .*= d2 .* d3
        for i=1:length(Qrows)
            Qvals[i] /= d2[Qrows[i]] * d2[Qcols[i]] * d3[Qrows[i]] * d3[Qcols[i]]
        end
        Qx = mul_Qx_COO!(Qx, Qrows, Qcols, Qvals, x)
        xTQx_2 =  x' * Qx / 2
        for i=1:length(Arows)
            Avals[i] /= d1[Arows[i]] * d2[Acols[i]] * d3[Acols[i]]
        end
        λ .*= d1
        ATλ = mul_ATλ_COO!(ATλ, Arows, Acols, Avals, λ)
        Ax = mul_Ax_COO!(Ax, Arows, Acols, Avals, x)
        b ./= d1
        c ./= d2 .* d3
        cTx = c' * x
        pri_obj = xTQx_2 + cTx + c0
        lvar .*= d2 .* d3
        uvar .*= d2 .* d3
        dual_obj = b' * λ - xTQx_2 + view(s_l,ilow)'*view(lvar,ilow) -
                    view(s_u,iupp)'*view(uvar,iupp) +c0

        s_l ./= d2 .* d3
        s_u ./= d2 .* d3
        rb .= Ax .- b
        rc .= ATλ .-Qx .+ s_l .- s_u .- c
#         rcNorm, rbNorm = norm(rc), norm(rb)
        rcNorm, rbNorm = norm(rc, Inf), norm(rb, Inf)
    end

    elapsed_time = time() - start_time

    stats = GenericExecutionStats(status, QM, solution = x[1:QM.meta.nvar],
                                  objective = pri_obj ,
                                  dual_feas = rcNorm,
                                  primal_feas = rbNorm,
                                  solver_specific = Dict(:multipliers => λ,
                                                         :multipliers_L => s_l,
                                                         :multipliers_U => s_u),
                                  iter = k,
                                  elapsed_time=elapsed_time)
    return stats
end

mehrotraPCQuadBounds (generic function with 1 method)

In [402]:
# probleme1
Q = [6. 2. 1.
     2. 5. 2.
     1. 2. 4.]
c = [-8.; -3; -3]
c0 = 0.
A = [1. 0. 1.
    0. 1. 1.]
b = [0.; 3]
lvar = [0.;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [405]:
# stats_mpc1 =  mehrotraPCQuadBounds(SM, scaling=true, max_iter=20)
@code_warntype mehrotraPCQuadBounds(SM, scaling=true, max_iter=20)
println(stats_mpc1)

Variables
  #unused#::Core.Compiler.Const(var"#mehrotraPCQuadBounds##kw"(), false)
  @_2::NamedTuple{(:scaling, :max_iter),Tuple{Bool,Int64}}
  @_3::Core.Compiler.Const(mehrotraPCQuadBounds, false)
  QM0::QuadraticModel
  max_iter::Int64
  ϵ_pdd::Float64
  ϵ_rb::Float64
  ϵ_rc::Float64
  tol_Δx::Float64
  ϵ_μ::Float64
  max_time::Float64
  scaling::Bool
  display::Bool
  @_14::Int64
  @_15::Float64
  @_16::Float64
  @_17::Float64
  @_18::Float64
  @_19::Float64
  @_20::Float64
  @_21::Bool
  @_22::Bool

Body::GenericExecutionStats
1 ── %1  = Base.haskey(@_2, :max_iter)::Core.Compiler.Const(true, false)
│          %1
│          (@_14 = Base.getindex(@_2, :max_iter))
└───       goto #3
2 ──       Core.Compiler.Const(:(@_14 = 200), false)
3 ┄─       (max_iter = @_14)
│    %7  = Base.haskey(@_2, :ϵ_pdd)::Core.Compiler.Const(false, false)
└───       goto #5 if not %7
4 ──       Core.Compiler.Const(:(@_15 = Base.getindex(@_2, :ϵ_pdd)), false)
└───       Core.Compiler.Const(:(goto %12), false

In [366]:
function createQuadraticModel(qpdata; name="qp_pb")
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals,
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, name=name)
end

path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
path_pb_QP = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_marosmeszaros"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib\\AFIRO.SIF"

In [406]:
qpdata2 = readqps(afiro)  # obj   -4.6475314286E+02
# presolve_QP!(qpdata2)
QM2 = createQuadraticModel(qpdata2)
SM2 = SlackModel(QM2);
SM2.meta

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: qp_pb-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   



In [407]:
stats_mpc2 =  mehrotraPCQuadBounds(SM2, max_iter=50)
println(stats_mpc2)

float32  3
iter float32 = 5
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -464.75314285562973
  primal feasibility: 4.405364961712621e-13
  dual feasibility: 1.6047801665308725e-7
  solution: [79.99999999996542  25.499999999812836  54.500000000152575  84.7999999999638 ⋯ 104.22545313511188]
  iterations: 9
  elapsed time: 0.023999929428100586
  solver specific:
    multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 2.0166785348864863e-13]
    multipliers_L: [5.405661415704836e-13  5.105555453459115e-12  6.700906792065277e-13  5.099578416540255e-13 ⋯ 0.0]
    multipliers: [-0.6285714285555644  2.5043004636521094e-11  -0.3447714285408298  -0.22857142855378737 ⋯ -5.281624690225728e-12]


┌ Info:   iter       obj      rgap      ‖rb‖      ‖rc‖      ‖Δx‖     α_pri      α_du         μ  
└ @ Main In[401]:128
┌ Info:      0   6.4e+02   2.6e+01   5.0e+02   8.3e+00   0.0e+00   0.0e+00   0.0e+00   8.1e+02
└ @ Main In[401]:133
┌ Info:      1  -5.5e+01   9.3e+01   5.3e+01   1.7e+00   4.2e+02   8.9e-01   8.1e-01   1.5e+02
└ @ Main In[392]:180
┌ Info:      2  -5.1e+01   2.6e+01   7.1e-03   2.2e-01   2.7e+02   1.0e+00   8.8e-01   3.0e+01
└ @ Main In[392]:180
┌ Info:      3  -1.7e+02   2.6e+00   4.1e-04   7.1e-02   2.1e+02   1.0e+00   7.1e-01   1.0e+01
└ @ Main In[392]:180
┌ Info:      4  -4.2e+02   1.1e-01   1.8e-04   5.7e-02   5.1e+02   7.0e-01   9.9e-01   2.9e+00
└ @ Main In[392]:180
┌ Info:      5  -4.6e+02   2.6e-03   9.2e-05   1.7e-02   6.5e+01   1.0e+00   7.7e-01   6.0e-01
└ @ Main In[392]:180
┌ Info:      6  -4.6e+02   3.4e-03   2.9e-05   1.1e-03   1.4e+01   8.5e-01   1.0e+00   4.1e-02
└ @ Main In[392]:180
┌ Info:      7  -4.6e+02   3.4e-05   3.7e-07   1.3e-05   2.0e+00   1.0

In [375]:
@benchmark mehrotraPCQuadBounds(SM2, max_iter=50, display=false)

BenchmarkTools.Trial: 
  memory estimate:  249.70 KiB
  allocs estimate:  2458
  --------------
  minimum time:     463.000 μs (0.00% GC)
  median time:      562.800 μs (0.00% GC)
  mean time:        581.197 μs (2.95% GC)
  maximum time:     4.643 ms (86.05% GC)
  --------------
  samples:          8557
  evals/sample:     1

In [408]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\BNL2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3)
#SM3 = SlackModel(QM3)
QM3.meta

┌ Info: Using 'BNL2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJ' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 11294)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: qp_pb
   All variables: ████████████████████ 3489   All constraints: ████████████████████ 2324  
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ████████████████████ 3489             lower: █████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 515   
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: █████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 482   
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████⋅⋅⋅⋅⋅⋅⋅⋅ 1327  
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 2324  
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 99.83% sparsity)   13999 



In [409]:
stats_mpc3 = mehrotraPCQuadBounds(QM3, scaling=true,max_iter=50);
println(stats_mpc3)

┌ Info:   iter       obj      rgap      ‖rb‖      ‖rc‖      ‖Δx‖     α_pri      α_du         μ  
└ @ Main In[401]:128
┌ Info:      0   1.9e+06   2.2e+00   9.3e+04   8.6e+01   0.0e+00   0.0e+00   0.0e+00   1.9e+05
└ @ Main In[401]:133
┌ Info:      1   1.1e+06   3.6e+00   4.7e+04   4.2e+01   6.0e+04   4.9e-01   5.8e-01   9.5e+04
└ @ Main In[392]:180
┌ Info:      2   7.0e+05   9.2e+00   2.6e+04   1.5e+01   3.8e+04   4.5e-01   6.8e-01   5.7e+04
└ @ Main In[392]:180


error 3
iter float32 = 3
error 8


┌ Info:      3   3.9e+05   1.7e+01   1.2e+04   8.4e+00   3.8e+04   5.5e-01   8.0e-01   2.7e+04
└ @ Main In[392]:180
┌ Info:      4   3.0e+05   2.1e+01   7.1e+03   3.7e+00   4.3e+04   3.9e-01   6.1e-01   2.0e+04
└ @ Main In[392]:180
┌ Info:      5   1.8e+05   2.4e+01   3.2e+03   6.7e-01   4.3e+04   5.5e-01   8.0e-01   9.7e+03
└ @ Main In[392]:180
┌ Info:      6   9.4e+04   2.9e+01   1.1e+03   1.5e-01   8.3e+04   6.6e-01   8.0e-01   3.7e+03
└ @ Main In[392]:180
┌ Info:      7   7.2e+04   3.2e+01   6.0e+02   8.7e-02   2.0e+05   4.5e-01   4.6e-01   2.4e+03
└ @ Main In[392]:180
┌ Info:      8   4.6e+04   3.0e+01   2.9e+02   2.2e-02   1.7e+05   5.1e-01   7.5e-01   1.2e+03
└ @ Main In[392]:180
┌ Info:      9   3.5e+04   2.4e+01   1.7e+02   3.8e-01   4.2e+03   4.1e-01   8.3e-01   7.5e+02
└ @ Main In[392]:180
┌ Info:     10   2.5e+04   2.8e+01   7.0e+01   2.2e-01   2.5e+04   6.0e-01   9.5e-01   4.3e+02
└ @ Main In[392]:180
┌ Info:     11   1.8e+04   2.4e+01   4.0e+01   7.0e-02   2.7e+04   4.3e-

Generic Execution stats
  status: maximum iteration
  objective value: 1811.3136828706315
  primal feasibility: 5.296969770444565e-8
  dual feasibility: 0.04707526709672294
  solution: [41.933333332988184  41.93333333298818  33.06666666693188  33.06666666693188 ⋯ 768.0000507233757]
  iterations: 50
  elapsed time: 0.46700000762939453
  solver specific:
    multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 1.7288270745261942e-13]
    multipliers_L: [1.986415059549727e-12  1.986415059549727e-12  2.519062645237696e-12  2.519062645237697e-12 ⋯ 0.0]
    multipliers: [6.37303415051358e-10  1.890558035176251e-9  1.1672475147575582e-9  2.845763136156321e-10 ⋯ -1.539089627112997e-15]


In [283]:
# problems AGG2, BNL2, FFFFF800, CYCLE, D2Q06C, FIT1P, FIT2P, CRE-B, SHIP04S, SHIP08L, LOTFI, MODSZK1
# CONT-050, CONT-100, CONT-101, CONT-201, CONT-300, BOYD2, LISWET1, POWELL20, QSEBA,QGROW22,QFFFFF80, QSHIP12L
# QGFRDXPN, QPCBOEI2, CVXQP3_L, CVXQP1_L
pb4 = string(path_pb_QP, "\\STADAT1.SIF")
qpdata4 = readqps(pb4)
QM4 = createQuadraticModel(qpdata4)
QM4.meta

┌ Info: Using 'STADAT1' as NAME (l. 1)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJ.FUNC' as objective (l. 17)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 10019)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'RANGES' as RANGES (l. 12020)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:552
┌ Info: Using 'BOUNDS' as BOUNDS (l. 14021)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


  Problem name: qp_pb
   All variables: ████████████████████ 2001   All constraints: ████████████████████ 3999  
            free: ████████████████████ 2001              free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ██████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 1999  
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ███████████⋅⋅⋅⋅⋅⋅⋅⋅⋅ 2000  
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: ( 99.90% sparsity)   2000            linear: ████████████████████ 3999  
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 99.88% sparsity)   9997  



In [284]:
stats_mpc4 = mehrotraPCQuadBounds(QM4, max_iter=200, max_time=600, scaling=true)
println(stats_mpc4)

┌ Info:   iter       obj      rgap      ‖rb‖      ‖rc‖      ‖Δx‖     α_pri      α_du         μ  
└ @ Main In[276]:127
┌ Info:      0   0.0e+00   6.4e-01   5.5e-01   4.4e+05   0.0e+00   0.0e+00   0.0e+00   1.3e-04
└ @ Main In[276]:132
┌ Info:      1  -3.4e+02   9.3e-01   5.5e-01   4.4e+05   3.3e+00   3.6e-09   3.1e-08   3.8e-03
└ @ Main In[196]:173
┌ Info:      2  -2.9e+02   8.6e-01   5.5e-01   4.4e+05   3.1e+00   5.3e-09   1.2e-08   6.6e-03
└ @ Main In[196]:173
┌ Info:      3  -3.1e+02   8.7e-01   5.5e-01   4.4e+05   2.6e+00   7.0e-09   2.6e-09   7.0e-03
└ @ Main In[196]:173
┌ Info:      4  -4.1e+01   1.4e-01   5.5e-01   4.4e+05   3.2e+00   9.4e-09   6.1e-09   7.8e-03
└ @ Main In[196]:173
┌ Info:      5  -1.7e+02   7.1e-01   5.5e-01   4.4e+05   3.3e+00   8.0e-09   4.7e-09   8.4e-03
└ @ Main In[196]:173
┌ Info:      6  -2.8e+02   7.7e-01   5.5e-01   4.4e+05   3.3e+00   3.8e-08   1.7e-08   1.0e-02
└ @ Main In[196]:173
┌ Info:      7  -1.5e+02   5.5e-01   5.5e-01   4.4e+05   2.9e+00   6.0

┌ Info:     82  -1.8e+07   7.5e-01   7.4e-02   2.0e+04   4.6e+03   5.8e-01   6.7e-01   5.1e+00
└ @ Main In[196]:173
┌ Info:     83  -1.8e+07   7.5e-01   7.8e-02   1.6e+04   9.2e+02   1.3e-01   1.5e-01   5.1e+00
└ @ Main In[196]:173
┌ Info:     84  -2.1e+07   6.8e-01   8.0e-02   3.4e+04   1.6e+04   8.0e-01   4.3e-01   4.6e+00
└ @ Main In[196]:173
┌ Info:     85  -2.1e+07   6.8e-01   7.7e-02   3.4e+04   6.3e+02   7.9e-02   1.4e-02   4.5e+00
└ @ Main In[196]:173
┌ Info:     86  -2.1e+07   6.8e-01   8.1e-02   3.4e+04   2.2e+02   4.6e-03   7.0e-03   4.7e+00
└ @ Main In[196]:173
┌ Info:     87  -2.1e+07   6.8e-01   8.1e-02   3.2e+04   7.9e+02   2.0e-01   6.3e-02   4.8e+00
└ @ Main In[196]:173
┌ Info:     88  -2.1e+07   6.8e-01   8.4e-02   2.9e+04   9.4e+02   7.5e-02   9.0e-02   5.4e+00
└ @ Main In[196]:173
┌ Info:     89  -2.1e+07   6.8e-01   8.8e-02   2.9e+04   4.3e+01   1.6e-04   3.4e-04   6.0e+00
└ @ Main In[196]:173
┌ Info:     90  -2.0e+07   7.1e-01   8.4e-02   2.2e+04   5.5e+03   8.2e-

iter float32 = 200

┌ Info:    194  -1.8e+07   7.3e-01   1.1e-01   4.3e+04   3.2e+04   6.8e-01   2.4e-01   2.2e+01
└ @ Main In[196]:173
┌ Info:    195  -1.9e+07   7.2e-01   1.0e-01   4.1e+04   3.0e+03   3.1e-01   2.8e-02   2.1e+01
└ @ Main In[196]:173
┌ Info:    196  -1.8e+07   7.5e-01   1.1e-01   3.3e+04   5.8e+03   3.5e-01   3.3e-01   1.8e+01
└ @ Main In[196]:173
┌ Info:    197  -1.7e+07   7.6e-01   1.1e-01   1.6e+04   3.5e+03   8.2e-01   6.1e-01   6.8e+00
└ @ Main In[196]:173
┌ Info:    198  -1.7e+07   7.6e-01   1.0e-01   1.7e+04   1.1e+03   8.5e-02   1.1e-01   6.7e+00
└ @ Main In[196]:173
┌ Info:    199  -1.8e+07   7.6e-01   2.7e-01   1.7e+04   8.9e+02   8.9e-02   7.4e-04   7.4e+00
└ @ Main In[196]:173
┌ Info:    200  -1.8e+07   7.4e-01   4.3e-01   1.6e+04   3.3e+03   2.3e-01   3.1e-03   7.9e+00
└ @ Main In[196]:173



Generic Execution stats
  status: maximum iteration
  objective value: -1.8357099516362395e7
  primal feasibility: 19.413831595605906
  dual feasibility: 520.8575058335878
  solution: [-2111.457487700329  -2111.480915267614  -2111.5035939556674  -2111.5260313810822 ⋯ 23.22558368808788]
  iterations: 200
  elapsed time: 3.7360000610351562
  solver specific:
    multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 4.2979243643980025]
    multipliers_L: [0.0  0.0  0.0  0.0 ⋯ 0.10509853315550517]
    multipliers: [4.903114503747794  2.5509604804453554  1.8317639789962152  1.4888745174914746 ⋯ -4.41738920705498]


## Test sur plusieurs problèmes

In [378]:
problems = []
i_max = 14
i = 1
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"; "DTOC3.SIF";
                                                         "HS35MOD.SIF";"QBORE3D.SIF";
                                                        "QCAPRI.SIF"; "QETAMACR.SIF";
                                                          "QFORPLAN.SIF"; "QPCSTAIR.SIF";
                                                        "QPCSTAIR.SIF"; "QPILOTNO.SIF";
                                                        "QRECIPE.SIF"; "QSHELL.SIF";
                                                        "QSIERRA.SIF"; "QSTAIR.SIF";
                                                        "QSTANDAT.SIF"; "UBH1.SIF"; 
                                                        "YAO.SIF"]) # problems with fixed variables
        

        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF";
                        "EXDATA.SIF"; "QFORPLAN.SIF"; "QGFRDXPN.SIF"; "VALUES.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end
        push!(problems, createQuadraticModel(qpdata_i, name=file_name[1:end-4]))
        
        if i==i_max
            break
        end
        i += 1
    end
end


25FV47.SIF
ADLITTLE.SIF
AFIRO.SIF
AGG.SIF


┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Lecont

AGG2.SIF
AGG3.SIF
BANDM.SIF
BEACONFD.SIF
BLEND.SIF

┌ Info: Using 'AGG' as RHS (l. 1852)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 536)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 2878)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG3' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 536)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 2884)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'BANDM' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\pa


BNL1.SIF
BNL2.SIF
BOEING1.SIF
BOEING2.SIF


┌ Info: Using 'BLEND' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'C' as objective (l. 98)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '' as RHS (l. 376)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'BNL1' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJ' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 3994)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'BNL2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJ' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSRead

BRANDY.SIF


┌ Info: Using 'OBJECTIV' as objective (l. 23)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS1' as RHS (l. 898)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'RANGE1' as RANGES (l. 919)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:552
┌ Info: Using 'INTBOU' as BOUNDS (l. 930)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622
┌ Info: Using 'BRANDY' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '10000A' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 1378)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [379]:
problems_stats = solve_problems(mehrotraPCQuadBounds, problems, 
                                colstats=[:name, :status, :elapsed_time, :objective, :dual_feas, :primal_feas])

error 12
error 29
error 85
error 60
error 63


┌ Info:            Name           status      Time      f(x)      Dual    Primal  
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\XEfsx\src\bmark\run_solver.jl:67
┌ Info:          25FV47          unknown   2.3e-01   6.3e+03   1.2e-02   1.9e-05
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\XEfsx\src\bmark\run_solver.jl:83
┌ Info:        ADLITTLE       acceptable   1.0e-03   2.3e+05   1.3e-04   6.3e-11
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\XEfsx\src\bmark\run_solver.jl:83
┌ Info:           AFIRO       acceptable   1.0e-03  -4.6e+02   1.6e-07   4.4e-13
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\XEfsx\src\bmark\run_solver.jl:83


error 35


┌ Info:             AGG         max_iter   7.3e-02  -3.6e+07   8.0e-01   4.3e-02
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\XEfsx\src\bmark\run_solver.jl:83


error 155


...

error 34
error 70
error 33
error 41


┌ Info:            BNL2          unknown   4.9e-01   2.7e+03   3.6e-02   6.8e-04
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\XEfsx\src\bmark\run_solver.jl:83


error 15


┌ Info:         BOEING1         max_iter   1.2e-01  -3.3e+02   3.5e+26   8.9e-03
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\XEfsx\src\bmark\run_solver.jl:83
┌ Info:         BOEING2       acceptable   5.0e-03  -3.2e+02   4.0e-06   8.6e-10
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\XEfsx\src\bmark\run_solver.jl:83
┌ Info:          BRANDY       acceptable   1.7e-02   1.5e+03   2.1e-04   3.5e-10
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\XEfsx\src\bmark\run_solver.jl:83


,id,name,nvar,ncon,nequ,status,objective,elapsed_time,iter
,Int64,String,Int64,Int64,Int64,Symbol,Float64,Float64,Int64
1,1,25FV47,1571,821,0,unknown,6315.32,0.233,85
2,2,ADLITTLE,97,56,0,acceptable,225495.0,0.000999928,15
3,3,AFIRO,32,27,0,acceptable,-464.753,0.000999928,9
4,4,AGG,163,488,0,max_iter,-3.59885e7,0.073,200
5,5,AGG2,302,516,0,max_iter,-2.02386e7,0.247,200
6,6,AGG3,302,516,0,max_iter,1.03424e7,0.255,200
7,7,BANDM,472,305,0,acceptable,-158.628,0.00699997,17
8,8,BEACONFD,262,173,0,acceptable,33592.5,0.0079999,30
9,9,BLEND,83,74,0,acceptable,-30.8121,0.00100017,11
